<div style="text-align:right;color:blue">version id: __VERSION_ID__</div>

# General Principles of Debugging

As you will probably have realised by now, a lot of time allocated for programming is not actually spent on writing code, but rather on finding and eliminating errors or ''bugs'', as they are known in computer science. To spend our time productively, we have to develop good strategies for avoiding and locating these bugs (once we have found a bug, fixing it is often relatively straightforward). Fortunately there exist general guidelines and tools that help us with this. In this notebook we will review some of these and apply them to a particular example.

<hr style="height: 2px">

### What you will learn
In this notebook we will cover the following topics:

* General principles of debugging
* The scientific method
* Application of the scientific method to debug an example code

<hr style="height: 2px">

*&#169; Eike Mueller, University of Bath 2019-2024. These lecture notes are copyright of Eike Mueller, University of Bath. They are provided exclusively for educational purposes at the University and are to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

## What is a bug?
The term ''bug'' for an error in a computer code was coined in 1945, when computer engineers found a moth trapped in the circuits of the Mark II computer. This poor insect had shortcircuited the machine, causing an error in program execution.

![First bug: moth in computer relay of the Mark II computer](./figures/FirstBug.png)

Of course, if our code does not work nowadays this is usually not due to some mechanical failure but rather due to some mistake in the computer code. In semester 1 we discussed how thoroughly testing code can avoid these mistakes. Unfortunately, it is still virtually impossible to write bug-free code and we need to come up with strategies for identifying errors efficiently and reliably.

### Finding bugs
Obviously, it helps greatly if we can localise the issue with our code and narrow down the problem to a few lines of suspect code. For this, writing modular code is essential. In practice this means:

* splitting our program into (well documented) functions
* using well defined interfaces with clear input and output
* not using global variables
* using object oriented design (we will talk about this later this semester)


However, even when we have limited the problem to a handful of lines, it is still very hard to spot bugs by just looking at some computer code. Since ''the eye sees what the brain wants to see'', it is easy to miss a mistake in code that we think should work. This has two important consequences that might sound obvious but are often ignored:

1. Code is not correct because we **think** it is correct
2. Correctness can only be judged based on **evidence**

### Digression: the scientific method
To illustrate the importance of the second point, let us look at a seemingly unrelated problem: the development of new scientific theories. To consider a particular example, recall that before the Renaissance one of the leading astronomical theories assumed that celestial bodies such as the stars, the planets and their moons were attached to large, concentric crystal spheres that enclose the Earth. These theories predicted the movements of the planets very well, so there were good reasons to assume that the theory was correct. Obviously, we now know that the universe is not made up of crystal spheres, but how would we be able to prove this just by looking at the trajectories of celestial bodies? While observing the moons of Jupiter (which, incidentially, have inspired the logo of project Jupyter...) Galileo Galilei discovered that these moons sometimes disappeared behind the planet. Since the crystal spheres were deemed to be indestructible, this clearly conflicts with the old theory. To address this shortcoming, one could form a new hypothesis: the movement of the planets and their moons is governed by gravitational attraction. Note that Galilei's thinking when ''debugging'' the incorrect theory went like this:

1. **(old) hypothesis**: celestial bodies are attached to crystal spheres
2. **design experiment** to test the hypothesis: if this is true, then the moons of Jupiter can not disappear
3. **carry out experiment to gather evidence**: my telescope shows me that the moons do disappear
4. **correct old hypothesis** and form **new hypothesis**: the crystal sphere theory must be wrong, planets and moons move according to gravitational attraction

It now seems inconceivable to us not to apply this approach, but before Galilei it was completely acceptable to ignore the evidence (and instead rely on the authority of ancient philosophical texts). Galilei's new approach is known as the **Scientific Method** and it can be summarised as follows:

*''A method of procedure that has characterized natural science since the 17th century, consisting in systematic **observation**, **measurement**, and **experiment**, and the **formulation, testing, and modification of hypotheses**.''*

### Debugging with the Scientific Method
Suprisingly, when debugging code it is equally common to ignore the evidence, which makes the process much harder than it needs to be. Thoughts like the following are much more common *''I **believe** this loop is correct, therefore my code must produce the right result. The bug must be in some other part of the code.''*. 

Consider the following example: we want to write a function to compute the sum

$$
\text{reciprocal\_sum}(n) = \sum_{j=0,2,4,\dots,2n} \frac{1}{3^j}
$$

(the summation variable $j$ takes on all even values between $0$ and $2n$).

Here is a simple Python implementation:

In [ ]:
def reciprocal_sum(n):
    r_sum = 0
    for j in range(0,2*n,2):
        s_j = 1//3**j
        r_sum += s_j
    return r_sum

Let's try to run the code for $n=1$:

In [ ]:
reciprocal_sum(2)

This must be wrong since $$\text{reciprocal\_sum}(2) = \frac{1}{3^0}+\frac{1}{3^2} + \frac{1}{3^4}=1+\frac{10}{81}>1.$$

Furthermore, we also get the wrong result for $n=0$:

In [ ]:
reciprocal_sum(0)

Thinking back to last semester, we would have spotted the issue by writing a set of accompanying tests such as:

In [ ]:
def test_reciprocal_sum_zero():
    assert reciprocal_sum(0) == 1
    
def test_reciprocal_sum_larger_than_one():
    assert reciprocal_sum(2) > 1
    
run_tests()

Let us try to apply the scientific method to debug the code:

#### Attempt \#1

1. **formulate hypothesis**: the loop bounds are correct
2. **design experiment**: if this is true, I should be able to see this by printing out $j$ in each iteration. This should print out all even numbers from $0$ to $16$ (inclusive). 
3. **carry out experiment**: add a print statement to the loop body:

```Python
def reciprocal_sum(n):
    r_sum = 0
    for j in range(0,2*n,2):
        print (j)
        s_j = 1//3**j        
        r_sum += s_j
    return r_sum
```
$\Rightarrow$ the code prints out $0,2,4,6,8,10,12,14$, i.e. $16$ is missing.

4. **correct hypothesis**: the loop bounds are **not** correct. We need to change the upper loop index from `2n` to `2n+1`. This then prints out all even numbers between $0$ and $16$ inclusive.

Let's fix the code and run the tests again:

In [ ]:
def reciprocal_sum(n):
    r_sum = 0
    for j in range(0,2*n+1,2):
        s_j = 1//3**j
        r_sum += s_j
    return r_sum

run_tests()

That looks better since we get the correct result for $n=0$. Unfortunately the code code still produces the wrong result if $n>0$, so it is time to apply the scientific method again:

#### Attempt \#2
1. **formulate hypothesis**: the individual terms in the sum are correct
2. **design experiment**: if this is true, I should be able to see this by printing out the variable `s_j`. For $j>1$ I would expect this variable to have a value between 0 and 1.
3. **carry out experiment**: add a print statement to the loop body:

```Python
def reciprocal_sum(n):
    r_sum = 0
    for j in range(0,2*n+1,2):        
        s_j = 1//3**j        
        print (j,s_j)
        r_sum += s_j
    return r_sum
```
$\Rightarrow$ for $j=0$ we get the correct result, but for $j>0$ `s_j=0`, which is clearly wrong.

4. **correct hypothesis**: Clearly something must be wrong with the way `s_j` is computed. We need to fix this in the code.

Looking at the code more closely, we find that `s_j` is computed by using integer division: `s_j=1//3**j`. For $j>1$ we have that $3^j>1$ and hence the result will be zero. We can fix this problem by replacing the line with `s_j=1/3**j` (only a single `/`). This results in the following code:

In [ ]:
def reciprocal_sum(n):
    r_sum = 0
    for j in range(0,2*n+1,2):
        s_j = 1/3**j        
        r_sum += s_j
    return r_sum

run_tests()

Now both tests pass.

Running this code for $n=1$ gives the correct result of $10/9\approx 1.111111\dots$:

In [ ]:
reciprocal_sum(1)

Observe that the two hypotheses that we formulated there **specific** and **verifyable**. A bad hypothesis would be: *''something is wrong in the for-loop''*. This is crucial to guarantee that we are able to localise the bug, i.e. identify the exact line which triggers it.

## Summary and outlook
Hopefully this little example demonstrates how a systematic approach to debugging can greatly simplify the process. For complex, real-life codes with thousands of lines it is the only way of homing in on the problem.

In the examples above we have gathered evidence simply by using print statements. This often works remarkably well and is a simple and robust method. However, there are better tools for this, and in the next notebook we will look at using the built-in debugger in JupyterLab. This allows us to easily step through the code line-by-line while inspecting the values of all variables simultaneously.